# predict-lack-of-url
skelley@air.org<br> 
sarora@air.org<br>
February 2019<br>

## Description
This notebook tries to explain bias in missing observations, which are mostly caused to a lack of an identified URL or if a URL exists, lack of successfully scraped website data. It takes as input a matrix of firm level measures produced by the R script, firm_level_patent_measures_v2.R

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import re
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import logit
import graphviz 

## Load and prep data
Here we are loading firm patent measures (also containing employee and url/website content data)

In [2]:
# general firm name cleaning fxn
def clean_firm_name (firm):
    firm_clnd = re.sub('(\.|,| corporation| incorporated| llc| inc| international| gmbh| ltd)', '', firm, flags=re.IGNORECASE).rstrip()
    return firm_clnd

In [55]:
firm_data = pd.read_csv('../../data/patents/measures/firm_level_measures.csv')
print (firm_data.columns.values)
print (firm_data.shape)
firm_data.head()

['firm' 'lookup_firm' 'lookup_firm_web' 'firm_length' 'name_clnd'
 'name_length' 'manual_fix' 'hit_url' 'hit_url_length' 'rank' 'matches'
 'public' 'acquired_merged' 'Votes' 'Group' 'nano' 'green' 'synbio'
 'count_all_ind' 'max_emps' 'size_state' 'sme' 'num_patents_all' 'pages'
 'num_words' 'avg_words_per_page' 'first_year' 'num_patents_3'
 'mean_assignees_all' 'mean_assignees_3' 'mean_inventors_all'
 'mean_inventors_3']
(1376, 32)


,firm,lookup_firm,lookup_firm_web,firm_length,name_clnd,name_length,manual_fix,hit_url,hit_url_length,rank,...,num_patents_all,pages,num_words,avg_words_per_page,first_year,num_patents_3,mean_assignees_all,mean_assignees_3,mean_inventors_all,mean_inventors_3
0,Hon Hai Precision Industry Co.,Hon Hai Precision Industry Co,Hon Hai Precision Industry Co,29.0,Foxconn - Official Site,23,NaN,foxconn.com/,23,2,...,17936,12.0,376119.0,31343.250000,1995,122.0,1.345339,1.959016,2.171387,3.647541
1,Tata Consultancy Services Limited,Tata Consultancy Services Limited,Tata Consultancy Services Limited,33.0,"Tata Consultancy Services | Technology, Digita...",61,NaN,tcs.com/,20,1,...,315,54.0,422808.0,7829.777778,2005,1.0,1.006349,1.000000,3.047619,2.000000
2,Hitachi,Hitachi,Hitachi,7.0,Hitachi - Official Site,23,NaN,hitachi.com/,23,1,...,38517,18.0,49138.0,2729.888889,1976,30.0,1.148843,1.148148,3.910066,3.740741
3,GENERAL ELECTRIC COMPANY,GENERAL ELECTRIC COMPANY,GENERAL ELECTRIC COMPANY,24.0,GE - Official Site,18,NaN,ge.com/,19,1,...,41692,19.0,325353.0,17123.842105,1976,26.0,1.005541,1.000000,2.624097,4.153846
4,Samsung Electronics,Samsung Electronics,Samsung Electronics,19.0,Electronics & Appliances: Tablets ... - Samsun...,50,NaN,samsung.com/us/,27,1,...,2,60.0,2701683.0,45028.050000,2001,NaN,2.000000,NaN,2.500000,NaN


<b>Model DVs</b>
1. whether employment data was successfully gathered
2. whether a firm URL was found
3. whether website content was crawled

<b>Explanatory variables</b>
1. the first year the firm patented
2. number of total patents assigned to the firm, logged
3. mean number of assignees with which the firm co-patented
4. mean inventors associated with their patents
5. the industry or industries in which the firm patents (synthetic biology, nanotechnology, and/or renewable energy);
6. the length of the firm’s name.  

In [56]:
# add y variables 
firm_data_with_ys = firm_data
firm_data_with_ys['has_emps'] = 1
firm_data_with_ys.loc[firm_data_with_ys['size_state'] == 'FirmSize.UNDEFINED', ['has_emps']] = 0

firm_data_with_ys['has_url'] = 1
firm_data_with_ys.loc[pd.isna(firm_data_with_ys['hit_url']), ['has_url']] = 0

firm_data_with_ys['has_web_content'] = 1
firm_data_with_ys.loc[pd.isna(firm_data_with_ys['pages']), ['has_web_content']] = 0

firm_data_with_ys.to_csv('../../data/analysis/measures/firm_data_with_ys.csv')

In [57]:
# add transformations / add'l X variables
firm_data_with_ys['log_num_patents_all'] = (firm_data_with_ys['num_patents_all'] + 1).apply(np.log)
firm_data_with_ys['log_max_emps'] = (firm_data_with_ys['max_emps'] + 1).apply(np.log)
firm_data_with_ys['num_industries'] = firm_data_with_ys['nano'] + firm_data_with_ys['synbio'] + firm_data_with_ys['green']

In [58]:
select_cols = [col for col in firm_data.columns if col in 
          ['has_emps', 'has_url', 'has_web_content', 'first_year', 'log_num_patents_all', 'mean_assignees_all', 'mean_inventors_all', 'num_industries', 'firm_length', 'log_max_emps']]
imputed_for_modeling = firm_data_with_ys[select_cols].fillna(0)
imputed_for_modeling.to_csv('../../data/patents/measures/imputed_for_modeling.csv')
imputed_for_modeling.head()

,firm_length,first_year,mean_assignees_all,mean_inventors_all,has_emps,has_url,has_web_content,log_num_patents_all,log_max_emps,num_industries
0,29.0,1995,1.345339,2.171387,1,1,1,9.794621,13.129341,2
1,33.0,2005,1.006349,3.047619,1,1,1,5.755742,12.886639,1
2,7.0,1976,1.148843,3.910066,1,1,1,10.558881,12.674722,3
3,24.0,1976,1.005541,2.624097,1,1,1,10.638089,12.653962,2
4,19.0,2001,2.000000,2.500000,1,1,1,1.098612,12.640274,1


In [59]:
imputed_for_modeling.shape

(1376, 10)

In [60]:
firm_data_with_ys.loc[(firm_data_with_ys['has_web_content'] == 1) & (firm_data_with_ys['has_emps'] == 1)].shape

(1132, 38)

## Logit

In [21]:
logit_url = logit("has_url ~ first_year + log_num_patents_all" 
                   " + mean_assignees_all + mean_inventors_all + firm_length"
                   " + num_industries + log_max_emps", imputed_for_modeling).fit()

logit_web_content = logit("has_web_content ~ first_year + log_num_patents_all" 
                   " + mean_assignees_all + mean_inventors_all + firm_length + "
                   " + num_industries + log_max_emps", imputed_for_modeling).fit()

Optimization terminated successfully.
         Current function value: 0.321154
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.475622
         Iterations 6


In [22]:
print(logit_url.summary())
print(logit_web_content.summary())

                           Logit Regression Results                           
Dep. Variable:                has_url   No. Observations:                 1489
Model:                          Logit   Df Residuals:                     1481
Method:                           MLE   Df Model:                            7
Date:                Thu, 26 Mar 2020   Pseudo R-squ.:                  0.1834
Time:                        16:42:57   Log-Likelihood:                -478.20
converged:                       True   LL-Null:                       -585.57
Covariance Type:            nonrobust   LLR p-value:                 8.646e-43
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.1388      0.862     -0.161      0.872      -1.828       1.550
first_year              0.0002      0.000      0.329      0.742      -0.001       0.001
log_num_patents_

## Decision tree

In [23]:
clf = tree.DecisionTreeClassifier()
grid = GridSearchCV(clf, {'max_depth' : [2, 3, 4, 5,6,10]}, scoring='f1')
xs = imputed_for_modeling.drop(['has_emps', 'has_url', 'has_web_content'], axis=1)
y = imputed_for_modeling['has_web_content']
grid.fit(xs, y)
grid.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 2}

In [24]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

In [25]:
clf = tree.DecisionTreeClassifier(random_state =42, max_depth=2)
clf.fit(xs, y)
precision = np.mean(cross_val_score(clf, xs, y, scoring = 'precision', cv=cv))
recall = np.mean(cross_val_score(clf, xs, y, scoring = 'recall', cv=cv))
accuracy = np.mean(cross_val_score(clf, xs, y, scoring = 'accuracy', cv=cv))
f1 = np.mean(cross_val_score(clf, xs, y, scoring = 'f1', cv=cv))

In [26]:
y.value_counts()

1    1190
0     299
Name: has_web_content, dtype: int64

In [27]:
print (recall)
print (accuracy)
print (f1)

0.9702076203070428
0.7856823266219239
0.8770610860899399


In [29]:
feature_imp = sorted(list(zip(xs, clf.feature_importances_)) , key=lambda x: x[1])
feature_imp

[('firm_length', 0.0),
 ('first_year', 0.0),
 ('mean_assignees_all', 0.0),
 ('mean_inventors_all', 0.0),
 ('num_industries', 0.0),
 ('log_num_patents_all', 0.15434728627268487),
 ('log_max_emps', 0.8456527137273151)]

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=list(xs.columns.values),   
                                filled=True, rounded=True, proportion=True, rotate=False, class_names=['no content','has content']) 
graph = graphviz.Source(dot_data) 
graph

In [31]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

The binary tree structure has 7 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 5] <= 1.7005987167358398 else to node 4.
	node=1 test node: go to node 2 if X[:, 4] <= 5.980482339859009 else to node 3.
		node=2 leaf node.
		node=3 leaf node.
	node=4 test node: go to node 5 if X[:, 5] <= 4.102609157562256 else to node 6.
		node=5 leaf node.
		node=6 leaf node.



In [ ]:
#sanity check
# Sarah -- where does preds come from? 
relevant_xs = xs[['firm_length', 'max_emps']]
relevant_xs['preds'] = preds

In [ ]:
relevant_xs[relevant_xs['max_emps'] < 2.5][relevant_xs['firm_length']<22.5].head()

In [104]:
relevant_xs[relevant_xs['max_emps'] < 2.5][relevant_xs['firm_length']>22.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
34,31.0,2.0,0.0
37,28.0,2.0,0.0
173,40.0,0.0,0.0
180,24.0,0.0,0.0
183,38.0,0.0,0.0


In [106]:
relevant_xs[2.5 < relevant_xs['max_emps']][relevant_xs['max_emps'] < 791.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
0,20.0,734.0,1.0
6,29.0,706.0,1.0
11,14.0,365.0,1.0
13,18.0,204.0,1.0
14,21.0,11.0,1.0


In [108]:
relevant_xs[relevant_xs['max_emps'] > 2.5][relevant_xs['max_emps']>791.5].head()

/Users/skelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,firm_length,max_emps,preds
1,32.0,60593.0,1.0
2,20.0,2141.0,1.0
4,6.0,100487.0,1.0
5,12.0,134800.0,1.0
7,16.0,99000.0,1.0
